In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs   #import map styles/types
import cartopy.feature as cfeature  # features such as the ocean, coastlines rivers, etc
from matplotlib import colorbar, colors
import cmocean
import cartopy as cp
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,4]
mpl.rcParams['figure.dpi'] = 350
mpl.rcParams['savefig.dpi'] = 350

mpl.rcParams['font.size'] = 20
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['lines.linewidth']= 2.0

In [2]:
import numpy as np
import xarray as xr

def pop_add_cyclic(ds):
    
    nj = ds.TLAT.shape[0] # size of POP grid
    ni = ds.TLONG.shape[1]

    xL = int(ni/2 - 1)
    xR = int(xL + ni)

    tlon = ds.TLONG.data
    tlat = ds.TLAT.data
#    print('Here')    
    tlon = np.where(np.greater_equal(tlon, min(tlon[:,0])), tlon-360., tlon) #make monotoncially increasing
    lon  = np.concatenate((tlon, tlon + 360.), 1) # concatenate to make larger array
    lon = lon[:, xL:xR] #restrict to middle rane
#    print('Here')   
    if ni == 320: # this is the x1 POP grid
        print('\n')
       # lon[367:-3, 0] = lon[367:-3, 0] + 360.        #####TUPLE PROBLEM IS HERE
      #  print('Here')
    lon = lon - 360.
    
    lon = np.hstack((lon, lon[:, 0:1] + 360.)) # add in cyclic point

    if ni == 320:
        print('\n')
        #lon[367:, -1] = lon[367:, -1] - 360.  #####TUPLE PROBLEM IS HERE
        
    #-- trick cartopy into doing the right thing:
    #   it gets confused when the cyclic coords are identical
    
   # lon[:, 0] = lon[:, 0] - 1e-8    ######TUPLE PROBLEM IS HERE
   
    #-- periodicity
    lat = np.concatenate((tlat, tlat), 1)
    lat = lat[:, xL:xR]
    lat = np.hstack((lat, lat[:,0:1]))

    TLAT = xr.DataArray(lat, dims=('nlat', 'nlon'))
    TLONG = xr.DataArray(lon, dims=('nlat', 'nlon'))
    
    dso = xr.Dataset({'TLAT': TLAT, 'TLONG': TLONG})
    
    # copy vars
    varlist = [v for v in ds.data_vars if v not in ['TLAT', 'TLONG']]
    for v in varlist:
        v_dims = ds[v].dims
        if not ('nlat' in v_dims and 'nlon' in v_dims):
            dso[v] = ds[v]
        else:
            # determine and sort other dimensions
            other_dims = set(v_dims) - {'nlat', 'nlon'}
            other_dims = tuple([d for d in v_dims if d in other_dims])
            lon_dim = ds[v].dims.index('nlon')
            field = ds[v].data
            field = np.concatenate((field, field), lon_dim)
            field = field[..., :, xL:xR]
            field = np.concatenate((field, field[..., :, 0:1]), lon_dim)       
            dso[v] = xr.DataArray(field, dims=other_dims+('nlat', 'nlon'), 
                                  attrs=ds[v].attrs)


    # copy coords
    for v, da in ds.coords.items():
        if not ('nlat' in da.dims and 'nlon' in da.dims):
            dso = dso.assign_coords(**{v: da})
                
            
    return dso

In [3]:
path = '/glade/u/home/chsharri/Work/NW/'
mask = xr.open_dataset(path+'/region_mask_nw.nc')

temp_150 = xr.open_dataset(path+'nw_ur_150_07.pop.h.TEMP.nc')

temp_cntrl_1 = xr.open_dataset(path+'nw_cntrl_03.pop.h.TEMP.nc')
temp_cntrl_2 = xr.open_dataset(path+'nw_cntrl_03m02.pop.h.TEMP.nc') 
temp_cntrl_3 = xr.open_dataset(path+'nw_cntrl_03m03.pop.h.TEMP.nc')

In [4]:
#####################################################################################################################################################
temp_150 = pop_add_cyclic(temp_150)
temp_cntrl_1 = pop_add_cyclic(temp_cntrl_1)
temp_cntrl_2 = pop_add_cyclic(temp_cntrl_2)
temp_cntrl_3 = pop_add_cyclic(temp_cntrl_3)

In [5]:
temp_cntrl_1 = temp_cntrl_1.TEMP[48:228,0,:,:]
temp_cntrl_2 = temp_cntrl_2.TEMP[48:228,0,:,:]
temp_cntrl_3 = temp_cntrl_3.TEMP[48:228,0,:,:]

temp_cntrl = (temp_cntrl_1 + temp_cntrl_2 + temp_cntrl_3) / 3

In [6]:
x,y = temp_cntrl_3['TLONG'], temp_cntrl_3['TLAT']

levels = np.arange(-30,32.5,2.5)


cmap = cmocean.cm.balance

norm = BoundaryNorm(levels, ncolors=cmap.N, clip= False)

fig,ax= plt.subplots(figsize =(19,13),subplot_kw=dict(projection=ccrs.Robinson(central_longitude=-60)))

ax.set_global()
ax.add_feature(cfeature.LAND, color = 'lightgray')
ax.add_feature(cfeature.COASTLINE)
ax.set_global()  


p = ax.pcolormesh(x, y,
                  vari, transform=ccrs.PlateCarree(), cmap = cmap, norm = norm)
t = np.arange(-30,32.5,5)

cbar = plt.colorbar(p, orientation='horizontal', pad=0.05, fraction=0.05,ax=ax, extend = 'min',ticks = t)
cbar.ax.tick_params(labelsize=23)
cbar.set_label('Max $\Delta$SST' + ' ($^o$C) ', size = 30)
    
    
ptitle = 'Maximum Monthly Sea Surface Temperature - 150 Tg'
land = ax.add_feature(cartopy.feature.NaturalEarthFeature('physical', 'land', '110m', linewidth=0.5,edgecolor='black', facecolor='lightgray'))    
  

#fig.savefig('/glade/work/vgarza/nw_figures/Max Delta SST.jpg' , bbox_inches='tight')

plt.show() 
plt.close()

KeyError: 'TLONG'